#### Needed Folder structure
```
───DON-011733
    │   DON-011733.yaml
    │   
    ├───20230306
    │   │   Image_001_001.raw
    │   │   DON-011733_20230306.yaml
    │   │   
    │   └───plane0
    │           F.npy
    │           Fneu.npy
    │           iscell.npy
    │           ops.npy
    │           spks.npy
    │           stat.npy
    │           
    └───day0
        │   Image_001_001.raw
        │   DON-011733_20230302.yaml
        │   
        └───plane0
                F.npy
                Fneu.npy
                iscell.npy
                ops.npy
                spks.npy
                stat.npy
```

#### Imports

In [4]:
# install needed packages
%pip install pyyaml==6.0 numpy==1.23.5 suite2p==0.12.1 parmap==1.6.0
%pip install matplotlib==3.7.1 scipy==1.10.1 tqdm==4.65.0 scikit-learn==1.2.2
%pip install pandas==2.0.1 networkx==3.1 opencv-python==4.7.0.72
#donatolabs

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# work with files 
import yaml
import copy

# do Math stuff
from itertools import permutations
import numpy as np
import sys
import os

from IPython.core.display import display, HTML


# Suite2p for TIFF file analysis
import suite2p
from suite2p.run_s2p import run_s2p, default_ops
from suite2p.registration import register

module_path = os.path.abspath(os.path.join('../'))
module_path = os.path.abspath(os.path.join('../../'))
sys.path.append(module_path)
from Multi_session_alignment_classes_functions import *
from manifolds.donlabtools.utils.calcium import calcium
from manifolds.donlabtools.utils.calcium.calcium import *

# usefull jupyter settings
%matplotlib inline
#%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

C:\Users\mauls\AppData\Local\Temp\ipykernel_33328\1202132604.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Autosaving every 180 seconds


In [3]:
!pip list

Package                       Version
----------------------------- ---------
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
asciitree                     0.3.3
asttokens                     2.2.1
attrs                         23.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
bcrypt                        4.0.1
beautifulsoup4                4.12.2
bleach                        6.0.0
bokeh                         3.1.1
cellpose                      2.2
certifi                       2023.7.22
cffi                          1.15.1
charset-normalizer            3.2.0
click                         8.1.3
cloudpickle                   2.2.1
colorama                      0.4.6
contourpy                     1.0.7
cryptography                  40.0.2
cycler                        0.11.0
dask                          2023.4.1
debugpy                       1.5.1
decorator 

## Setting

In [3]:
learning_mode = True

# Merging BMI sessions

## HowTo

In [4]:
# How it should be used (compact format)
# Load wanted animal data from root directory
if not learning_mode:
    root_dir = "D:\\Animals" 
    Animal.root_dir = "D:\\Animals" 
    plot_contour_pairs = False
    #day1_20230303
    #lastday_20230309
    wanted_animal_ids = ["DON-011733"] 
    wanted_session_ids = ["day0", "20230303", "20230309"] 
    #wanted_animal_ids = ["all"]
    #wanted_session_ids = ["all"]
    # only set restore to True, if you want to get original suite2p data instead of suite2p data based on merged mask
    animals = load_all(root_dir, wanted_animal_ids=wanted_animal_ids, restore=False,
                       wanted_session_ids=wanted_session_ids, print_loading=True)
    viz = Vizualizer(animals)
    for animal_id, animal in animals.items():
        # only set restore to True, if you want to recompute the merged session
        merged_session = animal.merge_sessions(restore=False, reference_session_id="day0",
                                               regenerate=True)

        # Plotting results
        viz.session_footprints(merged_session, figsize=(5,5))
        viz.session_contours(merged_session, figsize=(10,10), plot_center=False)
        viz.bursts(merged_session.animal_id, merged_session.session_id, fluorescence_type="F_detrended")

        # Plot contours of sessions in pairs of 2 for all combinations, with and without shift
        if plot_contour_pairs:
            sessions = animals[animal_id].sessions
            figsize=(20, 20)
            session_ids = list(sessions.keys())
            combinations = list(permutations(session_ids, 2))
            session_id_numbering = range(len(session_ids))
            numbering_combinations = list(permutations(session_id_numbering, 2))
            for num_combination, combination in zip(numbering_combinations, combinations):
                if num_combination[0] < num_combination[1]:
                    viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=False, figsize=figsize)
                    viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=True, figsize=figsize) 

## Detailed procedure for understanding 
- only using 3 sessions for faster computation

### Define yx-shift if not defined for every session

In [5]:
# Load wanted animal data from root directory
root_dir = "D:\\Animals" 
root_dir = "/scicore/projects/donafl00-calcium/Users/Sergej/Animals" 
Animal.root_dir = root_dir
#day1_20230303
#lastday_20230309
wanted_animal_ids = ["DON-011733"] #["all"]
wanted_session_ids = ["day0", "20230303", "20230309"] #["all"]
wanted_session_ids = ["all"]
animals = load_all(root_dir, wanted_animal_ids=wanted_animal_ids, restore=True,
                       wanted_session_ids=wanted_session_ids, print_loading=True)

Added animal: DON-011733
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/day0/DON-011733_20230302.yaml
Initialized session: DON-011733 day0
DON-011733
day0
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/day0/plane0
Getting cabincorr data from /scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/day0/plane0
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230303/DON-011733_20230303.yaml
Initialized updated session: DON-011733 20230303
Restored original suite2p files
DON-011733
20230303
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230303/plane0
Getting cabincorr data from /scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230303/plane0
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230309/DON-011733_20230309.yaml
Initialized session: DON-011733 20230309
DON-011733
20230309
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230309/plane0
Getting cabincorr data

In [6]:
#set reference session
animal_id = "DON-011733"
session_id = "day0" #"20230303"
animal = animals[animal_id]
sessions = animal.sessions
reference_session = sessions[session_id]
print(f"Reference session date: {reference_session.date}")

Reference session date: 20230302


In [7]:
# Define xy shift between intervidual sessions (only for intrasessional data)
for session_id, session in sessions.items():  
    session.set_yx_shift(reference_session)
    print(f"{session_id} shift: {session.yx_shift}")

20230303 shift: [7, 2]
20230309 shift: [-2, -1]
day0 shift: [0, 0]


### Merge sessions

In [8]:
# load original data
for session_id, session in sessions.items():
    if session.updated:
        backup_path_files(session.suite2p_path, restore=True) 
        backup_path_files(session.suite2p_path, restore=False)
        session.updated = False
        session.change_yaml_file("updated", False)
        session.c, session.contours, session.footprints = session.get_c_contours_footprints()

Restored original suite2p files
Backup path already exists. Skipping
DON-011733
20230303
/scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230303/plane0
Getting cabincorr data from /scicore/projects/donafl00-calcium/Users/Sergej/test/DON-011733/20230303/plane0


In [ ]:
# create a master mask by
# merging masks of every session, 
# by shifting and rotating cells, remove abroad cells and deduplicate
merger = Merger()
merged_stat = merger.merge_stat(sessions, reference_session, parallel = True)
print(f"Number of cells after merging: {merged_stat.shape[0]}")

In [ ]:
updated_sessions = {}
merged_unit_id = ""
Animal.root_dir = root_dir
for session_id, session in sessions.items():
    if session.session_type=="merged": #session_id in updated_sessions or :
        continue
    print(f"Updating session {session_id}")
    # shift and rotate merged mask and update suite2p files in session
    merger.shift_update_session_s2p_files(session, merged_stat)
    session.change_yaml_file("updated", True)
    updated_sessions[session_id] = Session(session.yaml_file_path,
                                           session.animal_id,
                                           print_loading=True)
    updated_sessions[session_id].pday = session.pday
    updated_sessions[session_id].load_data(restore=False, regenerate=True)
#animal.sessions = updated_sessions

In [11]:
# concatenate S2P results
#merged_s2p_path = merger.merge_s2p_files(updated_sessions, merged_stat) #FIXME: this is the correct code
merged_s2p_path = merger.merge_s2p_files(sessions, merged_stat) # only testing

In [ ]:
# create session object with merged session data
merged_yaml_path = merger.merge_yaml_metadata(sessions, reference_session)
merged_session = Session(merged_yaml_path,
                         animal_id=animal.animal_id, 
                         print_loading=True)
merged_session.load_data(restore=False, regenerate=True)
animal.sessions["merged"] = merged_session

In [ ]:
# plot standard footprints, contours, bursts
viz = Vizualizer(animals)
for animal_id, session_id, session in yield_animal_session(animals):
    if session.session_type == "merged":
        continue
    viz.session_footprints(session, figsize=(5,5))
    viz.session_contours(session, figsize=(10,10), plot_center=False)
    #viz.bursts(session.animal_id, session.session_id, fluorescence_type="F_filtered")
    traces = viz.bursts(session.animal_id, session.session_id, fluorescence_type="F_detrended")

In [ ]:
# plot updated footprints, contours, bursts
viz = Vizualizer(animals)
for session_id, session in updated_sessions.items():
    viz.session_footprints(session, figsize=(5,5))
    viz.session_contours(session, figsize=(10,10), plot_center=False)
    #viz.bursts(session.animal_id, session.session_id, fluorescence_type="F_filtered")
    traces = viz.bursts(session.animal_id, session.session_id, fluorescence_type="F_detrended")

In [ ]:
viz = Vizualizer(animals)
viz.session_footprints(merged_session, figsize=(5,5))
viz.session_contours(merged_session, figsize=(10,10), plot_center=False)
#viz.bursts(merged_session.animal_id, merged_session.session_id, fluorescence_type="F_filtered")
traces = viz.bursts(merged_session.animal_id, merged_session.session_id, fluorescence_type="F_detrended")

## More details about the procedure + visualizations

### Define yx-shift for every session + Visualization

#### Set yx-shift 

In [ ]:
"""Only for understanding purposes. The correct procedure is using shown in section HowTo"""
#define reference image
animal_id = "DON-011733"
session_id = "day0"#"20230303"
animal = animals[animal_id]
sessions = animal.sessions 
reference_session = sessions[session_id]

In [ ]:
"""Only for understanding purposes. The correct procedure is using shown in section HowTo"""
viz = Vizualizer()
viz.session_footprints(reference_session, figsize=(5,5))
viz.session_contours(reference_session, figsize=(10,10), plot_center=False)

In [ ]:
"""Only for understanding purposes. The correct procedure is using shown in section HowTo"""
# Calculating yx_shifts for all sessions
for i, s in sessions.items():
    s.refImg = None
refImg = reference_session.get_reference_image(n_frames_to_be_acquired=1000)
refAndMasks = register.compute_reference_masks(refImg, reference_session.ops)

# Plot reference Images
for session_id, sess in sessions.items():
    if session_id == reference_session.session_id:
        continue   
    sess.get_reference_image()
    
viz.multi_session_refImg(sessions, num_images_x=3)


#### Visualize shifted, rotated Data

In [ ]:
#Plot session contour shift combinations for own Algorithm and for human evaluated values
viz = Vizualizer()
figsize=(20, 20)
session_ids = list(sessions.keys())
combinations = list(permutations(session_ids, 2))
session_id_numbering = range(len(session_ids))
numbering_combinations = list(permutations(session_id_numbering, 2))
for num_combination, combination in zip(numbering_combinations, combinations):
    if num_combination[0] < num_combination[1]:
        if "merged" in combination:
            colors = ["white"]*len(session_ids)
            colors[num_combination[0]] = "red"
            colors[num_combination[1]] = "blue"
            viz.multi_session_contours(sessions=sessions, 
                                       combination=combination,
                                       colors=colors,
                                       plot_center=True, 
                                       transparent=True, 
                                       shift=True, 
                                       figsize=figsize)

In [ ]:
#Plot session contour shift combinations for own Algorithm and for human evaluated values
viz = Vizualizer()
figsize=(20, 20)
session_ids = ["day0", "merged"]# list(sessions.keys())
combinations = list(permutations(session_ids, 2))
session_id_numbering = range(len(session_ids))
numbering_combinations = list(permutations(session_id_numbering, 2))
for num_combination, combination in zip(numbering_combinations, combinations):
    if num_combination[0] < num_combination[1]:
        viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=False, figsize=figsize)
        #viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=True, figsize=figsize)

In [ ]:
"""Only for understanding purposes. The correct procedure is using shown in section HowTo"""
#Plot session contour shift combinations for human evaluated values
viz = Vizualizer()
figsize=(20, 20)
session_ids = list(sessions.keys())
combinations = list(permutations(session_ids, 2))
session_id_numbering = range(len(session_ids))
numbering_combinations = list(permutations(session_id_numbering, 2))
for num_combination, combination in zip(numbering_combinations, combinations):
    if num_combination[0] < num_combination[1]:
        viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=False, figsize=figsize)
        viz.multi_session_contours(sessions, combination=combination, plot_center=True, shift=True, figsize=figsize)
        if "day0" in combination or "merged" in combination:
            # without roation
            viz.multi_session_contours(sessions, combination=combination, plot_center=True, 
                                    shift=True, 
                                    yx_shift = None,
                                    rot_angle = 0,
                                    rot_center_yx = None,
                                    figsize=figsize,
                                    comment="without rotation")                           
            # Without shift
            viz.multi_session_contours(sessions, combination=combination, plot_center=True, 
                                    shift=True, 
                                    yx_shift = [0, 0],
                                    rot_angle = None,
                                    rot_center_yx = None,
                                    figsize=figsize,
                                    comment="without shift")

# for every session: take the first 100 frames create a .gif and afterwards a .mp4 video
if False:
    for session_id, session in sessions.items():
        l = Binary_loader()
        num_frames = 100
        binary_frames = l.load_binary(session.session_dir, num_frames)
        save_dir = os.path.join(root_dir, "figures")
        gif_name = f"{session.animal_id}_{session.session_id}_0-{num_frames}.gif"
        ani = l.binary_frames_to_gif(binary_frames, frame_range=[0, num_frames], save_dir=save_dir, comment=f"{session.animal_id}_{session.session_id}")
        gif_to_mp4(os.path.join(save_dir, gif_name))

### Merge all Subsessions to create full animal fluorescence data

Ask me (Sergej) for explanation. Had not enough time to create this section properly.
1. create master mask called `merged_stat`
   1. merge stat files of every session
   2. remove abroad cells
   3. deduplicate
2. update suite2p files for every session based on `merged_stat`
   1. shift merged_stat to individual yx_shifts of session
   2. rerun suite2p based on shifted `merged_stat`
   3. rerun cabincorr
3. Create merged Session
   1. create a new session folder called `merged`
   2. concatenate updated suite2p files of every session and save it in `merged`
   3. load merged session data with `session.get_session_data`